<a href="https://colab.research.google.com/github/AIM-Harvard/aimi_alpha/blob/main/aimi/lungmask/notebooks/lungmask_mwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ModelHub - Lung and Lung Lobes Segmentation**

This notebook provides an example of how to run an end-to-end (cloud-based) data analysis using the LungMask lung and lung lobes segmentation (robust to the presence of severe pathologies) model.

The way all the operations are executed - from pulling data to data postprocessing and the standardisation of the results - have the goal of promoting transparency and reproducibility.

## **Environment Setup**

This demo notebook is intended to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`.

From the dropdown menu under `Hardware accelerator`, select `GPU`. Let's check the Colab instance is indeed equipped with a GPU.

In [ ]:
import os
import sys
import shutil

import yaml

import time
import tqdm


# useful information
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))

print("\nCurrent directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

print("Python version    :", sys.version.split('\n')[0])

Thu Sep 29 21:32:58 2022

Current directory : /content
Hostname          : 16054438eab9
Username          : root
Python version    : 3.7.14 (default, Sep  8 2022, 00:06:44) 


The authentication to Google is necessary to run BigQuery queries.

Every operation throughout the whole notebook (BigQuery, fetching data from the IDC buckets) is completely free. The only thing that is needed in order to run the notebook is the set-up of a Google Cloud project. In order for the notebook to work as intended, you will need to specify the name of the project in the cell after the authentication one.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import files
from google.cloud import storage
from google.cloud import bigquery as bq

# INSERT THE ID OF YOUR PROJECT HERE!
project_id = "idc-sandbox-000"

Throughout this Colab notebook, for image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation. We will be running Plastimatch using the simple [PyPlastimatch](https://github.com/AIM-Harvard/pyplastimatch/tree/main/pyplastimatch) python wrapper. 

In [ ]:
%%capture
!apt install plastimatch

In [ ]:
# check plastimatch was correctly installed
!plastimatch --version

plastimatch version 1.7.0


---

Start by cloning the AIMI hub repository on the Colab instance.

The AIMI hub repository stores all the code we will use for pulling, preprocessing, processing, and postprocessing the data for this use case - as long as the others shared through AIMI hub.

In [ ]:
!git clone https://github.com/AIM-Harvard/aimi_alpha.git aimi

Cloning into 'aimi'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 409 (delta 17), reused 29 (delta 9), pack-reused 362
Receiving objects: 100% (409/409), 3.23 MiB | 29.05 MiB/s, done.
Resolving deltas: 100% (211/211), done.


To organise the DICOM data in a more common (and human-understandable) fashion after downloading those from the buckets, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). 

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
data/raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

In [ ]:
!git clone https://github.com/pieper/dicomsort dicomsort

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (63/63), done.


We will also use DCMQI for converting the resulting segmentation into standard DICOM SEG objects.

In [ ]:
%%capture
dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.2.4/dcmqi-1.2.4-linux.tar.gz"
dcmqi_download_path = "/content/dcmqi-1.2.4-linux.tar.gz"
dcmqi_path = "/content/dcmqi-1.2.4-linux"

!wget -O $dcmqi_download_path $dcmqi_release_url

!tar -xvf $dcmqi_download_path

!mv $dcmqi_path/bin/* /bin

---

In [ ]:
%%capture
!pip install pyplastimatch ipywidgets
!pip install git+https://github.com/JoHof/lungmask

In [ ]:
import shutil
import random

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk
import pyplastimatch as pypla

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

%matplotlib inline
%config InlineBackend.figure_format = "png"

import ipywidgets as ipyw

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:, -1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.Greens
my_greens = cmap(np.arange(cmap.N))
my_greens[:, -1] = np.linspace(0, 1, cmap.N)
my_greens = ListedColormap(my_greens)

cmap = plt.cm.Blues
my_blues = cmap(np.arange(cmap.N))
my_blues[:, -1] = np.linspace(0, 1, cmap.N)
my_blues = ListedColormap(my_blues)

cmap = plt.cm.spring
my_spring = cmap(np.arange(cmap.N))
my_spring[:, -1] = np.linspace(0, 1, cmap.N)
my_spring = ListedColormap(my_spring)
## ----------------------------------------

import seaborn as sns

Python version               :  3.7.14 (default, Sep  8 2022, 00:06:44) 
Numpy version                :  1.21.6


Provided everything was set up correctly, we can run the BigQuery query and get all the information we need to download the testing data from the IDC platform.

For this specific use case, we are going to be working with the NSCLC-Radiomics collection (Chest CT scans of lung cancer patients, with manual delineation of various organs at risk).

In [ ]:
%%bigquery --project=$project_id cohort_df

SELECT
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
FROM
  `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
WHERE
  StudyInstanceUID IN (
    SELECT
      StudyInstanceUID
    FROM
      `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
    WHERE
      StudyInstanceUID IN (
        SELECT
          StudyInstanceUID
        FROM
          `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
        WHERE
          (
            LOWER(
              dicom_pivot_v11.SegmentedPropertyTypeCodeSequence
            ) LIKE LOWER('80891009:SCT')
          )
        GROUP BY
          StudyInstanceUID
        INTERSECT DISTINCT
        SELECT
          StudyInstanceUID
        FROM
          `bigquery-public-data.idc_v11.dicom_pivot_v11` dicom_pivot_v11
        WHERE
          (
            dicom_pivot_v11.collection_id IN ('Community', 'nsclc_radiomics')
          )
        GROUP BY
          StudyInstanceUID
      )
    GROUP BY
      StudyInstanceUID
  )
GROUP BY
  dicom_pivot_v11.PatientID,
  dicom_pivot_v11.collection_id,
  dicom_pivot_v11.source_DOI,
  dicom_pivot_v11.StudyInstanceUID,
  dicom_pivot_v11.SeriesInstanceUID,
  dicom_pivot_v11.SOPInstanceUID,
  dicom_pivot_v11.gcs_url
ORDER BY
  dicom_pivot_v11.PatientID ASC,
  dicom_pivot_v11.collection_id ASC,
  dicom_pivot_v11.source_DOI ASC,
  dicom_pivot_v11.StudyInstanceUID ASC,
  dicom_pivot_v11.SeriesInstanceUID ASC,
  dicom_pivot_v11.SOPInstanceUID ASC,
  dicom_pivot_v11.gcs_url ASC

In [ ]:
# this works as intended only if the BQ query parses data from a single dataset
# if not, feel free to set the name manually!
dataset_name = cohort_df["collection_id"].values[0]

dataset_name

'nsclc_radiomics'

In [ ]:
# create the directory tree
!mkdir -p data models

!mkdir -p data/raw 
!mkdir -p data/raw/tmp data/raw/$dataset_name
!mkdir -p data/raw/$dataset_name/dicom

!mkdir -p data/processed
!mkdir -p data/processed/$dataset_name
!mkdir -p data/processed/$dataset_name/nii
!mkdir -p data/processed/$dataset_name/dicomseg

!mkdir -p data/model_input/
!mkdir -p data/lungmask_output/

## **Parsing Cohort Information from BigQuery Tables**

We can check the various fields of the table we populated by running the BigQuery query.

This table will store one entry for each DICOM file in the dataset (therefore, expect thousands of rows!)

In [ ]:
pat_id_list = sorted(list(set(cohort_df["PatientID"].values)))

print("Total number of unique Patient IDs:", len(pat_id_list))

display(cohort_df.info())

display(cohort_df.head())

Total number of unique Patient IDs: 127
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15678 entries, 0 to 15677
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          15678 non-null  object
 1   collection_id      15678 non-null  object
 2   source_DOI         15678 non-null  object
 3   StudyInstanceUID   15678 non-null  object
 4   SeriesInstanceUID  15678 non-null  object
 5   SOPInstanceUID     15678 non-null  object
 6   gcs_url            15678 non-null  object
dtypes: object(7)
memory usage: 857.5+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.2.276.0.7230010.3.1.3.2323910823.11504.15972...,1.2.276.0.7230010.3.1.4.2323910823.11504.15972...,gs://idc-open-cr/eff917af-8a2a-42fe-9e12-22bce...
1,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1004190115743500844746...,gs://idc-open-cr/f8cbf725-621d-4e18-8326-41789...
2,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1031280376053401623619...,gs://idc-open-cr/c73b3d12-78b1-4456-9a88-91ba2...
3,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1075071405629330534974...,gs://idc-open-cr/48b4ae0a-6936-44b4-a6bd-27c92...
4,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1125363119759695902111...,gs://idc-open-cr/3c36a30a-630b-4183-b87d-8a238...


---

## **Set Run Parameters**

Here below, we specify some of the variables we will use for the LungMask inference step and conversion to DICOM SEG.


In [ ]:
# FIXED PARAMETERS
data_base_path = "/content/data"
raw_base_path = "/content/data/raw/tmp"
sorted_base_path = os.path.join("/content/data/raw/", dataset_name, "dicom")

processed_base_path = os.path.join("/content/data/processed/", dataset_name)
processed_nifti_path = os.path.join(processed_base_path, "nii")

processed_dicomseg_path = os.path.join(processed_base_path, "dicomseg")

model_input_folder = "/content/data/model_input/"
model_output_folder = "/content/data/lungmask_output/"

dicomseg_json_path = "/content/aimi/aimi/lungmask/config/dicomseg_metadata.json"

## **Running the Analysis for a Single Patient**

In [ ]:
import aimi.aimi as aimi

from aimi import general_utils as aimi_utils
from aimi import lungmask as aimi_model

The following cells run all the processing pipeline, from pre-processing to post-processing.

In [ ]:
pat_id = random.choice(cohort_df["PatientID"].values)
pat_df = cohort_df[cohort_df["PatientID"] == pat_id].reset_index(drop = True)

display(pat_df.info())
display(pat_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          114 non-null    object
 1   collection_id      114 non-null    object
 2   source_DOI         114 non-null    object
 3   StudyInstanceUID   114 non-null    object
 4   SeriesInstanceUID  114 non-null    object
 5   SOPInstanceUID     114 non-null    object
 6   gcs_url            114 non-null    object
dtypes: object(7)
memory usage: 6.4+ KB


None

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,LUNG1-103,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.9509124927869378907016...,1.2.276.0.7230010.3.1.3.2323910823.17256.15972...,1.2.276.0.7230010.3.1.4.2323910823.17256.15972...,gs://idc-open-cr/4cf0beed-2347-40a8-b72e-de993...
1,LUNG1-103,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.9509124927869378907016...,1.3.6.1.4.1.32722.99.99.1750581825639617227311...,1.3.6.1.4.1.32722.99.99.1091769666371917878218...,gs://idc-open-cr/31a2404b-b7df-485f-899e-daa6b...
2,LUNG1-103,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.9509124927869378907016...,1.3.6.1.4.1.32722.99.99.2880931354100048642323...,1.3.6.1.4.1.32722.99.99.1022379727386505311442...,gs://idc-open-cr/4c1f46d4-bbba-4704-aff1-82a3e...
3,LUNG1-103,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.9509124927869378907016...,1.3.6.1.4.1.32722.99.99.2880931354100048642323...,1.3.6.1.4.1.32722.99.99.1088680118050829150328...,gs://idc-open-cr/3ba83262-122d-4d77-8bb8-1d8fa...
4,LUNG1-103,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.9509124927869378907016...,1.3.6.1.4.1.32722.99.99.2880931354100048642323...,1.3.6.1.4.1.32722.99.99.1092341942886933763350...,gs://idc-open-cr/34666409-5569-4209-9ae3-bb90e...


In [ ]:
# init

print("Processing patient: %s"%(pat_id))

patient_df = cohort_df[cohort_df["PatientID"] == pat_id]

dicomseg_fn = pat_id + "_SEG.dcm"

input_nifti_fn = pat_id + ".nii.gz"
input_nifti_path = os.path.join(model_input_folder, input_nifti_fn)

Processing patient: LUNG1-103


In [ ]:
# data cross-loading
aimi_utils.gcs.download_patient_data(raw_base_path = raw_base_path,
                                     sorted_base_path = sorted_base_path,
                                     patient_df = patient_df,
                                     remove_raw = True)

Copying files from IDC buckets to /content/data/raw/tmp/LUNG1-103...
Done in 8.3775 seconds.

Sorting DICOM files...
Done in 1.24017 seconds.
Sorted DICOM data saved at: /content/data/raw/nsclc_radiomics/dicom/LUNG1-103
Removing un-sorted data at /content/data/raw/tmp/LUNG1-103...
... Done.


In [ ]:
# DICOM CT to NIfTI - required for the processing
aimi_utils.preprocessing.pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                                 processed_nifti_path = processed_nifti_path,
                                                 pat_id = pat_id, verbose = True)


Running 'plastimatch convert' with the specified arguments:
  --input /content/data/raw/nsclc_radiomics/dicom/LUNG1-103/CT
  --output-img /content/data/processed/nsclc_radiomics/nii/LUNG1-103/LUNG1-103_CT.nii.gz
... Done.


In [ ]:
# prepare the `model_input` folder for the inference phase
aimi_utils.preprocessing.prep_input_data(processed_nifti_path = processed_nifti_path,
                                         model_input_folder = model_input_folder,
                                         pat_id = pat_id)

Copying /content/data/processed/nsclc_radiomics/nii/LUNG1-103/LUNG1-103_CT.nii.gz
to /content/data/model_input/LUNG1-103_0000.nii.gz...
... Done.


The following cell runs the DL-based segmentation.


In [ ]:
aimi_model.utils.processing.process_patient(pat_id = pat_id,
                                            model_input_folder = model_input_folder,
                                            model_output_folder = model_output_folder)

Done in 119.426 seconds.


---

In [ ]:
processed_seg_folder = os.path.join(processed_nifti_path, pat_id, "lungmask")

shutil.copytree(model_output_folder, processed_seg_folder)

'/content/data/processed/nsclc_radiomics/nii/LUNG1-103/lungmask'

In [ ]:
aimi_model.utils.postprocessing.nifti_to_dicomseg(sorted_base_path = sorted_base_path,
                                                  processed_base_path = processed_base_path,
                                                  dicomseg_json_path = dicomseg_json_path,
                                                  pat_id = pat_id)

---

In [ ]:
from pyplastimatch.utils import viz as viz_utils

In [ ]:
processed_nifti_path = os.path.join(processed_base_path, "nii")

ct_nifti_path = os.path.join(processed_nifti_path, pat_id, pat_id + "_CT.nii.gz")
lungmask_output_folder = os.path.join(processed_nifti_path, pat_id, "lungmask")

lungseg_nii_path = os.path.join(lungmask_output_folder, pat_id + "_lungseg.nii.gz")
lobeseg_nii_path = os.path.join(lungmask_output_folder, pat_id + "_lobeseg.nii.gz")

"""
alternative way of loading the resulting NIfTI files
nibabel can sometimes take better care of the orientation of the
converted/segmented images, but will orient the data differently by default
"""

#ct_nii = nib.load(ct_nii_path).dataobj
#seg_nii = nib.load(seg_nii_path).dataobj

ct_nii = sitk.GetArrayFromImage(sitk.ReadImage(ct_nifti_path))
lungseg_nii = sitk.GetArrayFromImage(sitk.ReadImage(lungseg_nii_path))
lobeseg_nii = sitk.GetArrayFromImage(sitk.ReadImage(lobeseg_nii_path))

In [ ]:
# class #1
sup_lobe_right_nii = np.copy(lobeseg_nii)
sup_lobe_right_nii[sup_lobe_right_nii != 1] = 0

inf_lobe_right_nii = np.copy(lobeseg_nii)
inf_lobe_right_nii[inf_lobe_right_nii != 2] = 0
inf_lobe_right_nii[inf_lobe_right_nii == 2] = 1

sup_lobe_left_nii = np.copy(lobeseg_nii)
sup_lobe_left_nii[sup_lobe_left_nii != 3] = 0
sup_lobe_left_nii[sup_lobe_left_nii == 3] = 1

mid_lobe_left_nii = np.copy(lobeseg_nii)
mid_lobe_left_nii[mid_lobe_left_nii != 4] = 0
mid_lobe_left_nii[mid_lobe_left_nii == 4] = 1

inf_lobe_left_nii = np.copy(lobeseg_nii)
inf_lobe_left_nii[inf_lobe_left_nii != 5] = 0
inf_lobe_left_nii[inf_lobe_left_nii == 5] = 1

In the next cell, we can visualise the result using a simple widget.

In [31]:
segmask_dict = dict()
segmask_cmap_dict = dict()

_ = viz_utils.AxialSliceSegmaskViz(ct_volume = ct_nii,
                                   segmask_dict = {"sup_lobe_right" : sup_lobe_right_nii,
                                                   "inf_lobe_right" : inf_lobe_right_nii,
                                                   "sup_lobe_left" : sup_lobe_left_nii,
                                                   "mid_lobe_left" : mid_lobe_left_nii,
                                                   "inf_lobe_left" : inf_lobe_left_nii},
                                   segmask_cmap_dict = {"sup_lobe_right" : my_greens,
                                                        "inf_lobe_right" : my_reds,
                                                        "sup_lobe_left" : my_greens,
                                                        "mid_lobe_left" : my_blues,
                                                        "inf_lobe_left" : my_reds},
                                   dpi = 100, figsize = (8, 8),
                                   min_hu = -1024, max_hu = 1024)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

---

## **Data Download**

In [ ]:
%%capture

archive_fn = "%s.zip"%(pat_id)

try:
  os.remove(archive_fn)
except OSError:
  pass

seg_dicom_path = os.path.join(processed_dicomseg_path, pat_id, dicomseg_fn)
ct_dicom_path = os.path.join(sorted_base_path, pat_id)

!zip -j -r $archive_fn $ct_dicom_path $seg_dicom_path

In [ ]:
filesize = os.stat(archive_fn).st_size/1024e03
print('Starting the download of "%s" (%2.1f MB)...\n'%(archive_fn, filesize))

files.download(archive_fn)

Starting the download of "LUNG1-103.zip" (27.4 MB)...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>